In [1]:
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt
import matplotlib

# ------------------------------------------------------------------------
# 1) Classify BGR -> Named color by Euclidean distance
# ------------------------------------------------------------------------
def get_color_name(bgr):
    """
    Given a BGR color tuple, determine which reference color
    (red, green, blue, cyan, magenta, yellow)
    it is closest to by Euclidean distance in BGR space.
    """
    reference_colors = {
        "red":     (0,   0, 255),
        "green":   (0, 255,   0),
        "blue":    (255,  0,   0),
        "cyan":    (255, 255,  0),
        "magenta": (255,  0, 255),
        "yellow":  (0, 255, 255)
    }

    distances = {}
    for name, ref_bgr in reference_colors.items():
        db = bgr[0] - ref_bgr[0]
        dg = bgr[1] - ref_bgr[1]
        dr = bgr[2] - ref_bgr[2]
        distances[name] = db*db + dg*dg + dr*dr

    return min(distances, key=distances.get)

# ------------------------------------------------------------------------
# 2) Create labeled outline, measure red intensities, and write results
# ------------------------------------------------------------------------
def create_smoothed_color_labeled_outline(
    stereo_img,
    output_directory,
    scale_factor=4.0,
    min_area=100,
    epsilon_factor=0.00001,
    max_spot_sum=266.74240094392,
    MAX_RED_INTENSITY=255.0
):
    """
    1) Upscale the image by 'scale_factor'.
    2) Apply Gaussian blur to each channel for smoother edges.
    3) Detect edges (Canny), then morphologically dilate->erode to close gaps.
    4) Find contours, filter out very small ones, and simplify with approxPolyDP.
    5) For each contour:
        - Find average color inside.
        - Determine nearest named color.
        - Calculate mean red, back-calc MSI mean, area, total MSI.
    6) Draw the outline in white on black, then convert black to transparent.
    7) Write all results to "signal measurement.txt."
    8) Return the RGBA outline and a list of measurements.
    """

    h, w = stereo_img.shape[:2]
    up_w = int(w * scale_factor)
    up_h = int(h * scale_factor)

    stereo_up = cv2.resize(
        stereo_img,
        (up_w, up_h),
        interpolation=cv2.INTER_LANCZOS4
    )

    b, g, r = cv2.split(stereo_up)
    b_blur = cv2.GaussianBlur(b, (5, 5), 0.7)
    g_blur = cv2.GaussianBlur(g, (5, 5), 0.7)
    r_blur = cv2.GaussianBlur(r, (5, 5), 0.7)

    edges_b = cv2.Canny(b_blur, 30, 90)
    edges_g = cv2.Canny(g_blur, 30, 90)
    edges_r = cv2.Canny(r_blur, 30, 90)

    edges = cv2.bitwise_or(edges_b, edges_g)
    edges = cv2.bitwise_or(edges, edges_r)

    kernel = np.ones((3, 3), np.uint8)
    edges = cv2.dilate(edges, kernel, iterations=1)
    edges = cv2.erode(edges, kernel, iterations=1)

    contours, _ = cv2.findContours(edges, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)

    filtered_contours = []
    for c in contours:
        area = cv2.contourArea(c)
        if area < min_area:
            continue
        epsilon = epsilon_factor * cv2.arcLength(c, True)
        c_simplified = cv2.approxPolyDP(c, epsilon, True)
        filtered_contours.append(c_simplified)

    outline_bgr = np.zeros((up_h, up_w, 3), dtype=np.uint8)

    short_names = {
        "red":     "rc",
        "green":   "gc",
        "blue":    "bc",
        "cyan":    "cc",
        "magenta": "mc",
        "yellow":  "yc"
    }
    color_counts = {}
    measurements = []
    intensity_data = []

    for contour in filtered_contours:
        mask = np.zeros((up_h, up_w), dtype=np.uint8)
        cv2.drawContours(mask, [contour], -1, 255, -1)

        mean_b_val = cv2.mean(b, mask=mask)[0]
        mean_g_val = cv2.mean(g, mask=mask)[0]
        mean_r_val = cv2.mean(r, mask=mask)[0]
        avg_bgr = (mean_b_val, mean_g_val, mean_r_val)

        color_name = get_color_name(avg_bgr)
        color_counts[color_name] = color_counts.get(color_name, 0) + 1
        cell_num = color_counts[color_name]
        short_label = f"{short_names.get(color_name, 'xx')}{cell_num}"

        cv2.drawContours(outline_bgr, [contour], -1, (255, 255, 255), 1)

        M = cv2.moments(contour)
        if M["m00"] != 0:
            cX = int(M["m10"] / M["m00"])
            cY = int(M["m01"] / M["m00"])
            cv2.putText(
                outline_bgr,
                short_label,
                (cX, cY),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.3,
                (255, 255, 255),
                1,
                cv2.LINE_AA
            )

        measurements.append((short_label, mean_r_val))

        back_calc_msi_mean = (mean_r_val / MAX_RED_INTENSITY) * max_spot_sum
        area_pixels = cv2.countNonZero(mask)
        back_calc_msi_total = back_calc_msi_mean * area_pixels

        intensity_data.append(
            (short_label, mean_r_val, back_calc_msi_mean, area_pixels, back_calc_msi_total)
        )

    outline_gray = cv2.cvtColor(outline_bgr, cv2.COLOR_BGR2GRAY)
    _, alpha = cv2.threshold(outline_gray, 1, 255, cv2.THRESH_BINARY)
    outline_rgba = cv2.cvtColor(outline_bgr, cv2.COLOR_BGR2BGRA)
    outline_rgba[:, :, 3] = alpha

    labeled_path = os.path.join(output_directory, "labeled_outline.png")
    cv2.imwrite(labeled_path, outline_rgba)
    print(f"[INFO] Labeled outline saved: {labeled_path}")

    signal_measurement_path = os.path.join(output_directory, "signal measurement.txt")
    with open(signal_measurement_path, 'w') as f:
        for (label, mean_r, back_mean, area_px, back_total) in intensity_data:
            f.write(
                f"{label}: mean red={mean_r:.2f}, "
                f"back-calc mean={back_mean:.2f}, "
                f"area(px)={area_px}, "
                f"back-calc total={back_total:.2f}\n"
            )
    print(f"[INFO] Saved signal measurements: {signal_measurement_path}")

    return outline_rgba, measurements

# ------------------------------------------------------------------------
# 3) Interactive point selection for image registration (optional)
# ------------------------------------------------------------------------
def select_points_with_counter(gray_image, title='Select Points'):
    """
    Displays the image and allows you to click points.
    Press ENTER or ESC when finished.
    Returns a list of (x, y) coordinates.
    """
    fig, ax = plt.subplots()
    ax.imshow(gray_image, cmap='gray')
    ax.set_title(title)

    coords = []
    def onclick(event):
        if event.inaxes == ax:
            ix, iy = event.xdata, event.ydata
            coords.append((ix, iy))
            ax.plot(ix, iy, 'r+', markersize=10, linewidth=2)
            fig.canvas.draw()

    cid = fig.canvas.mpl_connect('button_press_event', onclick)
    print("Click points. Press ENTER or ESC in the figure window when done.")
    plt.show()
    fig.canvas.mpl_disconnect(cid)
    plt.close(fig)

    return coords

# ------------------------------------------------------------------------
# 4) Main function (example usage)
# ------------------------------------------------------------------------
def main():
    Output_DIR = "/Users/yonghengwang/Downloads"

    # Load Cell_Boundary_Cell_Type.png
    stereo_img = cv2.imread("/Users/yonghengwang/Downloads/Cell_Boundary_Cell_Type.png")
    if stereo_img is None:
        raise ValueError("Could not read image at: /Users/yonghengwang/Downloads/Cell_Boundary_Cell_Type.png")

    outline_rgba, measurements = create_smoothed_color_labeled_outline(
        stereo_img,
        Output_DIR,
        scale_factor=4.0,
        min_area=500,
        epsilon_factor=0.00001,
        max_spot_sum=266.74240094392,
        MAX_RED_INTENSITY=250.0
    )

    # Load labeled outline and MSI image
    stereo_seq = cv2.imread("/Users/yonghengwang/Downloads/labeled_outline.png")
    msi = cv2.imread("/Users/yonghengwang/Downloads/MSI_v1.png")
    if stereo_seq is None or msi is None:
        print("One of the images for registration wasn't found. Skipping registration step.")
        return

    scale_factor_after_read = 4
    stereo_h, stereo_w = stereo_seq.shape[:2]
    msi_h, msi_w = msi.shape[:2]

    stereo_seq = cv2.resize(
        stereo_seq,
        (stereo_w * scale_factor_after_read, stereo_h * scale_factor_after_read),
        interpolation=cv2.INTER_CUBIC
    )
    msi = cv2.resize(
        msi,
        (msi_w * scale_factor_after_read, msi_h * scale_factor_after_read),
        interpolation=cv2.INTER_CUBIC
    )

    stereo_seq_gray = cv2.cvtColor(stereo_seq, cv2.COLOR_BGR2GRAY)
    msi_gray = cv2.cvtColor(msi, cv2.COLOR_BGR2GRAY)

    matplotlib.use('TkAgg')
    print("Select points on the labeled (stereo-seq) image.")
    moving_points = select_points_with_counter(
        stereo_seq_gray,
        title="Select Points on Upscaled Stereo-seq"
    )

    print("Select the SAME number of points on the MSI image.")
    fixed_points = select_points_with_counter(
        msi_gray,
        title="Select Points on Upscaled MSI"
    )

    if len(moving_points) != len(fixed_points):
        raise ValueError("Number of selected points must match.")

    moving_pts = np.array(moving_points, dtype=np.float32)
    fixed_pts = np.array(fixed_points, dtype=np.float32)

    M, inliers = cv2.estimateAffine2D(
        moving_pts,
        fixed_pts,
        method=cv2.RANSAC,
        ransacReprojThreshold=3.0
    )
    if M is None:
        raise ValueError("Could not estimate transform. Check your point selections.")

    H, W = msi_gray.shape
    registered_upscaled = cv2.warpAffine(stereo_seq, M, (W, H), flags=cv2.INTER_CUBIC)

    alpha = 0.4
    blended = cv2.addWeighted(msi, 1 - alpha, registered_upscaled, alpha, 0)

    reg_up_gray = cv2.cvtColor(registered_upscaled, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(reg_up_gray, 50, 150)
    color_edges = np.zeros_like(registered_upscaled)
    color_edges[edges > 0] = (255, 255, 255)
    final_image = cv2.addWeighted(blended, 1.0, color_edges, 0.8, 0)

    scale_factor_before_save = 4
    final_h, final_w = final_image.shape[:2]
    final_image_up = cv2.resize(
        final_image,
        (final_w * scale_factor_before_save, final_h * scale_factor_before_save),
        interpolation=cv2.INTER_CUBIC
    )

    out_name = "msi_with_stereo_labels_and_outlines_double_scaled.png"
    final_image_path = os.path.join(Output_DIR, out_name)
    cv2.imwrite(final_image_path, final_image_up)
    print(f"Saved final image to: {final_image_path}")

    # Display side-by-side
    fig, axes = plt.subplots(1, 2, figsize=(12, 5))
    axes[0].imshow(cv2.cvtColor(msi, cv2.COLOR_BGR2RGB))
    axes[0].set_title("Upscaled MSI")
    axes[0].axis("off")

    axes[1].imshow(cv2.cvtColor(final_image_up, cv2.COLOR_BGR2RGB))
    axes[1].set_title("Final (Labels & Outlines) - Further Upscaled")
    axes[1].axis("off")

    plt.tight_layout()
    plt.show()

if __name__ == "__main__":
    main()


[INFO] Labeled outline saved: /Users/yonghengwang/Downloads/labeled_outline.png
[INFO] Saved signal measurements: /Users/yonghengwang/Downloads/signal measurement.txt
Select points on the labeled (stereo-seq) image.
Click points. Press ENTER or ESC in the figure window when done.
Select the SAME number of points on the MSI image.
Click points. Press ENTER or ESC in the figure window when done.
Saved final image to: /Users/yonghengwang/Downloads/msi_with_stereo_labels_and_outlines_double_scaled.png
